In [ ]:
#!pip install Pillow

In [1]:
from PIL import Image, ImageDraw, ImageFont
import json
from urllib.request import urlopen
import requests
from io import BytesIO
from bs4 import BeautifulSoup

In [2]:
layout = Image.open("maket_plain.jpg") # макет должен лежать в той же папке, что и скрипт

In [3]:
base_config = json.load(open("base_config.json", 'r'))

In [4]:
def feedParser(fpath: str):
    
    # работает и со ссылками, и с локальными файлами    
    if fpath.startswith("http"):
        xmlSoup = BeautifulSoup(urlopen(fpath), "xml")
    else:
        xmlSoup = BeautifulSoup(open(fpath, 'r', encoding='utf-8'), "xml") 
        
    feedDict = dict()    
    
    for item in xmlSoup.find_all("item"):
        itemID = item.find("g:id").text
        iTitle = item.find("g:title").text
        iImageLink = item.find("g:image_link").text.strip()
        iBrand = item.find("g:brand").text.strip()
        iPrice = item.find("g:price").text.strip()
        iSalePrice = item.find("g:sale_price").text.strip() if item.find("g:sale_price") else ""
        
        feedDict[itemID] = {
            "ID": itemID,
            "title": iTitle,
            "image": iImageLink,
            "brand": iBrand,
            "price": iPrice,
            "sale_price": iSalePrice
        }
    
    return feedDict

In [5]:
feed = feedParser("") # don't forget to write the url

In [11]:
# config=config, font=font
def produceCreative(item: dict, base_layout=layout, base_config=base_config):
    layout = base_layout.copy()
    w = base_config["image"]["width"]
    h = base_config["image"]["height"]
    l = base_config["image"]["left"]
    t = base_config["image"]["top"]
    
    response = requests.get(item["image"])
    img = Image.open(BytesIO(response.content))#.convert("RGBA")
    img = img.resize((w,h), Image.ANTIALIAS)
      
    layout.paste(img, (l, t, l+img.width, t+img.height))
    
    text_features = [base_config[x] for x in base_config if x.startswith("text")]
    
    for TF in text_features:
        T = item[TF["feature"]]
        font = ImageFont.truetype("arial.ttf", TF["size"]) # относительная локальная ссылка на файл со шрифтом в формате .ttf
    
        draw_text = ImageDraw.Draw(layout)
        draw_text.text(
            (TF["left"], TF["top"]),
            T,
            fill = '#1C0606', # whatever color u like
            font=font,
        )
        
    layout.save(item["ID"] + ".jpg")
    layout.close()

In [7]:
produceCreative(feed[list(feed.keys())[11]])

In [12]:
for picID in list(feed.keys())[:40]:
    produceCreative(feed[picID])
    #print(picID)

33036245631075
32051545636963
32051563757667
32051526172771
13337050710115
32051526041699
33036244353123
33036244484195
32051527057507
32051525124195
32051563561059
32051561168995
32051563659363
32051564052579
33036229116003
12654753972323
12654819115107
12654819934307
33036232687715
31728406823011
31728438280291
31728726671459
31728752066659
32051546357859
31728562733155
31728595271779
31728610115683
32143975809123
32143981379683
32143983870051
32143966601315
32143968108643
32143970205795
32051563462755
33036214075491
33036214304867
32051536330851
32051563954275
32051546292323
32051540754531
